# Necessary Imports and Settings

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import xmltodict
import pickle
from collections import defaultdict
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

#from toy_dataset import contexts

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced", load_in_8bit=True)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /kuacc/users/bozyurt20/.conda/envs/hf/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so...


/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [65]:
class PathResult():
    def __init__(self, prompt, probability_correct, probability_wrong, max_prob):
        self.prompt = prompt
        self.probability_correct = probability_correct
        self.probability_wrong = probability_wrong
        self.max_prob = max_prob

In [73]:
context_previous = "John travelled to Oslo. Emma travelled to Sydney."

input_ids = tokenizer.encode(context_previous, return_tensors="pt").to(model.encoder.device)

len_input_ids = len(input_ids[0])
with torch.no_grad():
    out = model.encoder(input_ids, output_special=True)
    
special_hidden = out.special_hidden_states # 24 x (1, T, d)
special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
for i, hidden in enumerate(special_hidden):
    special_reformatted[i:i+1, :, :] = hidden

entities_hidden_states = special_reformatted[:, :-1, :]
entities_hidden_states = entities_hidden_states.permute(1, 0, 2)

new code working-modeling_t5


In [74]:
entities_hidden_states.shape

torch.Size([12, 24, 4096])

In [66]:
context_current = " Lucas was 30 years old. "
question_1 = "Where did John travel to?"
question_2 = "Where did Emma travel to?"
answer_1 = "Oslo"
answer_2 = "Sydney"
results = [];
def my_pipeline(prompt, entity_inds, entity_hidden_states, answer_correct, answer_wrong):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
    out = model.generate(
            input_ids=input_ids,
            entity_hidden_states=entity_hidden_states,
            entity_inds=entity_inds,
            max_new_tokens=1,
            return_dict_in_generate=True,
            output_scores=True
        ) 
    next_token_scores = torch.nn.functional.softmax(
                        out.scores[0].float(), dim=-1
                    )  # (batch_size * num_beams, vocab_size)
    probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
    probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()
    max_prob = next_token_scores[0,next_token_scores.argmax().item()].item()
    return probability_correct, probability_wrong, max_prob

entity_inds = list(range(12))
prompt = context_previous + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = context_previous + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(11))
entities_hidden_states_new = entities_hidden_states[:11, :, :]
prompt = context_previous[:-1] + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = context_previous[:-1] + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))



new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [69]:
entity_inds = list(range(10))
entities_hidden_states_new = torch.cat((entities_hidden_states[:5, :, :], entities_hidden_states[6:11, :, :]), dim=0)
prompt = "John travelled to Oslo Emma travelled to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John travelled to Oslo Emma travelled to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [75]:
entity_inds = list(range(9))
entities_hidden_states_new = torch.cat((entities_hidden_states[:3, :, :], entities_hidden_states[4:5, :, :], entities_hidden_states[6:11, :, :]), dim=0)
prompt = "John travelled Oslo Emma travelled to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John travelled Oslo Emma travelled to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(8))
entities_hidden_states_new = torch.cat((entities_hidden_states[:3, :, :], 
                                    entities_hidden_states[4:5, :, :], 
                                    entities_hidden_states[6:9, :, :],
                                    entities_hidden_states[10:11, :, :]), dim=0)
prompt = "John travelled Oslo Emma travelled Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John travelled Oslo Emma travelled Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [77]:
entity_inds = list(range(6))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], 
                                        entities_hidden_states[4:5, :, :],
                                        entities_hidden_states[6:9, :, :],
                                        entities_hidden_states[10:11, :, :]), dim=0)
prompt = "John Oslo Emma travelled Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John Oslo Emma travelled Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(4))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], 
                                        entities_hidden_states[4:5, :, :],
                                        entities_hidden_states[6:7, :, :],
                                        entities_hidden_states[10:11, :, :]), dim=0)
prompt = "John Oslo Emma Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John Oslo Emma Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [80]:
entity_inds = list(range(8))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], entities_hidden_states[3:5, :, :], entities_hidden_states[6:11, :, :]), dim=0)
prompt = "John to Oslo Emma travelled to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John to Oslo Emma travelled to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(6))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], 
                                    entities_hidden_states[3:5, :, :], 
                                    entities_hidden_states[6:7, :, :],
                                    entities_hidden_states[9:11, :, :]), dim=0)
prompt = "John to Oslo Emma to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John to Oslo Emma to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [82]:
entity_inds = list(range(5))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], 
                                        entities_hidden_states[4:5, :, :], 
                                        entities_hidden_states[6:7, :, :],
                                        entities_hidden_states[9:11, :, :]), dim=0)
prompt = "John Oslo Emma to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John Oslo Emma to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [84]:
entity_inds = list(range(7))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], entities_hidden_states[4:5, :, :], entities_hidden_states[6:11, :, :]), dim=0)
prompt = "John Oslo Emma travelled to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John Oslo Emma travelled to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(7))
entities_hidden_states_new = torch.cat((entities_hidden_states[:5, :, :], 
                                    entities_hidden_states[6:7, :, :], 
                                    entities_hidden_states[10:11, :, :]), dim=0)
prompt = "John travelled to Oslo Emma Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John travelled to Oslo Emma Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(7))
entities_hidden_states_new = torch.cat((entities_hidden_states[:1, :, :], 
                                        entities_hidden_states[3:5, :, :], 
                                        entities_hidden_states[6:9, :, :],
                                        entities_hidden_states[10:11, :, :]), dim=0)
prompt = "John to Oslo Emma travelled Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John to Oslo Emma travelled Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

entity_inds = list(range(7))
entities_hidden_states_new = torch.cat((entities_hidden_states[:3, :, :], 
                                        entities_hidden_states[4:5, :, :], 
                                        entities_hidden_states[6:7, :, :],
                                        entities_hidden_states[9:11, :, :]), dim=0)
prompt = "John travelled Oslo Emma to Sydney" + context_current + question_1
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_1, answer_2)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

prompt = "John travelled Oslo Emma to Sydney" + context_current + question_2
probability_correct, probability_wrong, max_prob = my_pipeline(prompt, entity_inds, entities_hidden_states_new, answer_2, answer_1)
results.append(PathResult(prompt, probability_correct, probability_wrong, max_prob))

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [85]:
for result in results:
    print(result.prompt)
    print(result.probability_correct)
    print(result.probability_wrong)
    print(result.max_prob)
    print()

John travelled to Oslo. Emma travelled to Sydney. Lucas was 30 years old. Where did John travel to?
0.2510606646537781
0.16400770843029022
0.2510606646537781

John travelled to Oslo. Emma travelled to Sydney. Lucas was 30 years old. Where did Emma travel to?
0.4167127311229706
0.01614193432033062
0.4167127311229706

John travelled to Oslo. Emma travelled to Sydney Lucas was 30 years old. Where did John travel to?
0.3200170695781708
0.09942752867937088
0.3200170695781708

John travelled to Oslo. Emma travelled to Sydney Lucas was 30 years old. Where did Emma travel to?
0.3432735204696655
0.01956755854189396
0.3432735204696655

John travelled to Oslo Emma travelled to Sydney Lucas was 30 years old. Where did John travel to?
0.23133867979049683
0.10086718946695328
0.23133867979049683

John travelled to Oslo Emma travelled to Sydney Lucas was 30 years old. Where did Emma travel to?
0.29553642868995667
0.015663813799619675
0.29553642868995667

John travelled Oslo Emma travelled to Sydney Lu

In [61]:
context_current = " Lucas was 30 years old. "
question_1 = "Where did John travel to?"
question_2 = "Where did Emma travel to?"

prompt_2 = context_previous + context_current + question_2

input_ids = tokenizer.encode(prompt_2, return_tensors="pt")
entity_inds = list(range(12))
out = model.generate(
        input_ids=input_ids,
        entity_hidden_states=entities_hidden_states,
        entity_inds=entity_inds,
        max_new_tokens=1,
        return_dict_in_generate=True,
        output_scores=True
    ) 
next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)

answer_correct = "Sydney"
answer_wrong = "Oslo"
probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

probability_correct, probability_wrong

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


(0.4167155921459198, 0.016142046079039574)

In [62]:
next_token_scores[0,next_token_scores.argmax().item()].item()

0.4167155921459198

In [49]:
context_current = " Lucas was 30 years old. "
question_1 = "Where did John travel to?"
question_2 = "Where did Emma travel to?"

prompt_1 = context_previous + context_current + question_1

input_ids = tokenizer.encode(prompt_1, return_tensors="pt")
entity_inds = list(range(12))
out = model.generate(
        input_ids=input_ids,
        #entity_hidden_states=entities_hidden_states,
        #entity_inds=entity_inds,
        max_new_tokens=1,
        return_dict_in_generate=True,
        output_scores=True
    ) 
next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)

answer_correct = "Oslo"
answer_wrong = "Sydney"
probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

probability_correct, probability_wrong

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


In [51]:
context_current = " Lucas was 30 years old. "
question_1 = "Where did John travel to?"
question_2 = "Where did Emma travel to?"

prompt_1 = context_current + question_1

input_ids = tokenizer.encode(prompt_1, return_tensors="pt")
entity_inds = list(range(12))
out = model.generate(
        input_ids=input_ids,
        #entity_hidden_states=entities_hidden_states,
        #entity_inds=entity_inds,
        max_new_tokens=1,
        return_dict_in_generate=True,
        output_scores=True
    ) 
next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)

answer_correct = "Oslo"
answer_wrong = "Sydney"
probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

probability_correct, probability_wrong

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


(0.00019699727999977767, 0.0060422383248806)

In [ ]:
template = tm11
context_current = "Lucas was 30 years old."

context_previous = "John travelled to Oslo. Emma travelled to Sydney."
char_1 = "John"
char_2 = "Emma"
answer_1 = "Oslo"
answer_2 = "Sydney"

question_1 = "Where did John travel to?"
question_2 = "Where did Emma travel to?"

entities_list = ["John", " ", "travelled", "to", "Oslo", ".", "Emma", " ", "travelled", "to", "Sydney", "."]

entity_hidden_states = previous_timestep(context_previous,  entities_list)

to_prepend = " ".join(entities_list)

prompt_current = to_prepend + " " + context_current

probability_correct_1, probability_wrong_1, _ = current_timestep_regular(prompt_current, question_1, answer_1, answer_2, template)
probability_correct_2, probability_wrong_2, _ = current_timestep_regular(prompt_current, question_2, answer_2, answer_1, template)

probability_correct_enhanced_1, probability_wrong_enhanced_1, _ = current_timestep_enhanced(prompt_current, 
                                                                      question_1, 
                                                                      answer_1, 
                                                                      answer_2,
                                                                      template, 
                                                                      entities_list, 
                                                                      entity_hidden_states)

probability_correct_enhanced_2, probability_wrong_enhanced_2, _ = current_timestep_enhanced(prompt_current, 
                                                                      question_2, 
                                                                      answer_2,
                                                                      answer_1,
                                                                      template, 
                                                                      entities_list, 
                                                                      entity_hidden_states)

improvement_correct_1 = probability_correct_enhanced_1 - probability_correct_1
improvement_correct_2 = probability_correct_enhanced_2 - probability_correct_2
improvement_wrong_1 = probability_wrong_enhanced_1 - probability_wrong_1
improvement_wrong_2 = probability_wrong_enhanced_2 - probability_wrong_2
improvements_correct_1.append( improvement_correct_1 )
improvements_correct_2.append( improvement_correct_2 )
improvements_wrong_1.append( improvement_wrong_1 )
improvements_wrong_2.append( improvement_wrong_2 )

#NOT RUN

if probability_correct_1 > probability_wrong_1:
    correct_guesses.append(1)
else:
    correct_guesses.append(0)
if probability_correct_enhanced_1 > probability_wrong_enhanced_1:
    correct_guesses.append(1)
else:
    correct_guesses.append(0) 

# Open-Ended Generation - Two chars two locs

In [3]:
def find_index_one(input_ids, entity_str, index):
    
    entity_id = tokenizer.encode(entity_str)
    
    if len(entity_id) != 2:
        print("Not an appropriate entity!")
        return
    
    entity_id = entity_id[0]
    
    input_ids_list = input_ids.tolist()

    all_entity_mention_indices = []
    for i, j in enumerate(input_ids_list[0]):
        if j == entity_id:
            all_entity_mention_indices.append(i)
    try:
        entity_ind = all_entity_mention_indices[index]
        return entity_ind
    except:
        print("entity not found in the input!")
        return

In [4]:
def previous_timestep(context, entities):
    
    input_ids = tokenizer.encode(context, return_tensors="pt")
    len_input_ids = len(input_ids[0])
    out = model.encoder(input_ids, output_special=True, output_hidden_states=True)
    special_hidden = out.special_hidden_states # 24 x (1, T, d)

    special_reformatted = torch.zeros(num_layers, len_input_ids, d_model) # (24, T, d)
    for i, hidden in enumerate(special_hidden):
        special_reformatted[i:i+1, :, :] = hidden
    
    entity_ind = find_index_one(input_ids, entities[0], 0)
    entities_hidden_states = special_reformatted[:, entity_ind, :].unsqueeze(0)

    for entity in entities[1:]:
        entity_ind = find_index_one(input_ids, entity, 0)
        entity_hidden_states = special_reformatted[:, entity_ind, :].unsqueeze(0)
        entities_hidden_states = torch.cat((entities_hidden_states,
                                       entity_hidden_states), dim=0)
    
    return entities_hidden_states

def current_timestep_regular(context, question, answer_correct, answer_wrong, template):
    
    prompt = template.render(context=context, question=question)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    out = model.generate(input_ids, max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)

    probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
    probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

    scores = []
    for i, score in enumerate(next_token_scores[0]):
        scores.append( (i, score.item()) )

    scores.sort(key=lambda x: x[1], reverse=True)
    scores = [(tokenizer.decode(a), b) for a, b in scores]

    return probability_correct, probability_wrong, scores

def current_timestep_enhanced(context, question, answer_correct, answer_wrong, template, entities, entity_hidden_states):
    
    prompt = template.render(context=context, question=question)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    entity_inds = []
    for entity in entities:
        entity_inds.append(find_index_one(input_ids, entity, 0))
        
    out = model.generate(
        input_ids=input_ids,
        entity_hidden_states=entity_hidden_states,
        entity_inds=entity_inds,
        max_new_tokens=1,
        return_dict_in_generate=True,
        output_scores=True
    ) 
    next_token_scores = torch.nn.functional.softmax(
                    out.scores[0].float(), dim=-1
                )  # (batch_size * num_beams, vocab_size)
    
    probability_correct = next_token_scores[0][tokenizer.encode(answer_correct)[0]].item()
    probability_wrong = next_token_scores[0][tokenizer.encode(answer_wrong)[0]].item()

    scores = []
    for i, score in enumerate(next_token_scores[0]):
        scores.append( (i, score.item()) )

    scores.sort(key=lambda x: x[1], reverse=True)
    scores = [(tokenizer.decode(a), b) for a, b in scores]

    return probability_correct, probability_wrong, scores


In [17]:
len(data_points)

625

In [20]:
template = tm11
context_current = "Lucas was 30 years old."
improvements_correct_1 = []
improvements_correct_2 = []
improvements_wrong_1 = []
improvements_wrong_2 = []
for data_point in data_points:
    
    char_1 = data_point.name_1
    char_2 = data_point.name_2
    answer_1 = data_point.city_1
    answer_2 = data_point.city_2
    context_previous = data_point.context
    
    question_1 = "Where did " + char_1 + " travel to?"
    question_2 = "Where did " + char_2 + " travel to?"
    
    entities_list = [char_1, char_2, answer_1, answer_2]
    
    entity_hidden_states = previous_timestep(context_previous,  entities_list)
    
    to_prepend = " ".join(entities_list)
    
    prompt_current = to_prepend + " " + context_current
    
    probability_correct_1, probability_wrong_1, _ = current_timestep_regular(prompt_current, question_1, answer_1, answer_2, template)
    probability_correct_2, probability_wrong_2, _ = current_timestep_regular(prompt_current, question_2, answer_2, answer_1, template)
    
    probability_correct_enhanced_1, probability_wrong_enhanced_1, _ = current_timestep_enhanced(prompt_current, 
                                                                          question_1, 
                                                                          answer_1, 
                                                                          answer_2,
                                                                          template, 
                                                                          entities_list, 
                                                                          entity_hidden_states)
    
    probability_correct_enhanced_2, probability_wrong_enhanced_2, _ = current_timestep_enhanced(prompt_current, 
                                                                          question_2, 
                                                                          answer_2,
                                                                          answer_1,
                                                                          template, 
                                                                          entities_list, 
                                                                          entity_hidden_states)
    
    improvement_correct_1 = probability_correct_enhanced_1 - probability_correct_1
    improvement_correct_2 = probability_correct_enhanced_2 - probability_correct_2
    improvement_wrong_1 = probability_wrong_enhanced_1 - probability_wrong_1
    improvement_wrong_2 = probability_wrong_enhanced_2 - probability_wrong_2
    improvements_correct_1.append( improvement_correct_1 )
    improvements_correct_2.append( improvement_correct_2 )
    improvements_wrong_1.append( improvement_wrong_1 )
    improvements_wrong_2.append( improvement_wrong_2 )
    
    #NOT RUN
    
    if probability_correct_1 > probability_wrong_1:
        correct_guesses.append(1)
    else:
        correct_guesses.append(0)
    if probability_correct_enhanced_1 > probability_wrong_enhanced_1:
        correct_guesses.append(1)
    else:
        correct_guesses.append(0) 

new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-modeling_t5
new code working-utils
new code working-m

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/predictions/holmes_two_char_two_locs.txt", "wb") as f:
    pickle.dump(results_dict, f)

In [ ]:
with open("/kuacc/users/bozyurt20/hpc_run/predictions/holmes_two_char_two_locs.txt", "rb") as f:
    results_dict = pickle.load(f)

# Templates

In [5]:
# Multiple Choice

tm1 = Template("""Read the following context and choose the best option to answer the question.
Context: {{ context }}
Question: {{ question }}
Options:
 - {{ answer_choices | join("\n - ") }}""")

tm2 = Template("""{{ context }}
{{ question }}
- {{ answer_choices | join("\n- ") }}""")

tm3 = Template("""{{ context }}
{{ question }}
Pick the correct answer from the following options:
- {{ answer_choices | join("\n- ") }}""")

tm4 = Template("""{{ context }}
According to the above context, choose the best option to answer the following question.
Question: {{ question }}
Options:
- {{answer_choices | join("\n - ")}}
""")

tm5 = Template("""{{ context }}
{{ question }}
Pick the best answer from the following options:
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

tm6 = Template("""{{ context }}
According to the above context, choose the best option to answer the following question.
Question: {{ question }}
Options:
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

tm7 = Template("""{{ context }}
{{ question }}
A. {{ answer0 }}
B. {{ answer1 }}
C. {{ answer2 }}
D. {{ answer3 }}""")

# Open-Ended

tm8 = Template("""Question: "{{question}}"
Context: "{{context}}"
Answer:""")

tm9 = Template("""{{ context }}
Given the paragraph above, please answer correctly the following
question:
{{ question }}""")

tm10 = Template("""Given the following passage
"{{context}}",
answer the following question. Note that the answer is present within
the text.
Question: {{question}}""")

tm11 = Template("{{context}} What is the answer to: {{question}}")

In [15]:
import sys
sys.path.append('/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/')
cp modeling_t5.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/models/t5/
cp utils.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/
cp modeling_outputs.py /kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/

SyntaxError: invalid syntax (676971914.py, line 3)